In [1]:
import os
import numpy as np

from openai import AzureOpenAI

from dotenv import load_dotenv
load_dotenv()

True

# Useful functions for completions:

In [75]:
def lp2p(p):
    return np.round(np.exp(p)*100,2)

def print_lp_info(completion):
    for i in completion.choices[0].logprobs.content:
        print(repr(f'Token: `{i.token}`, Logprob: {i.logprob} ({lp2p(i.logprob)}%)'))

def print_top_lp(completion):
    for i in completion.choices[0].logprobs.content:
        print(repr(f'Token: `{i.token}`, Logprob: {i.logprob} ({lp2p(i.logprob)}%)'))
        print("Top Logprobs:")
        for j in i.top_logprobs:
            print(repr(f"    `{j.token}`: {j.logprob} ({lp2p(j.logprob)}%)"))

def print_message(completion):
    print(completion.choices[0].message.content)

def extract_lp(completion):
    lps = []
    for i in completion.choices[0].logprobs.content:
        lps.append(i.logprob)
    return lps
    

# Model Call:

In [3]:
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION')
)

prompt = "You are a helpful assistant"
text = "Describe the universe in two words"

temperature = 0.2
max_tokens = 800
top_p = 0.95
frequency_penalty = 0
presence_penalty = 0

In [82]:
obs1 = 'Observed work party conducting hot work. Fire blankets and extinguishers were  in place. Gas test  was completed prior to work commencing along with the start work check. Fire watch was aware of the requirement to remain at the location for 30 mins  on completion of the task'
obs2 = 'Deck crew noticed some residual oil on the walkway causing a potential slip hazard. On investigation it was found to be coming from a hose used by Operations during night-shift at Chemical tank FTO 513.'

In [16]:
haz_prompt = 'You are a detail-oriented Process Safety Expert specializing in categorizing \nunsafe observations which have made hazards more likely to occur.\n\nYour task is to examine the provided observation and determine which of \nthe specified hazard categories have occurred, or were made more likely to occur,\nbecause of the act or condition described in the observation.\n\nAlways select at least one applicable hazard.\nIf no hazard seems applicable, select the hazard that is most likely to \noccur, based on the the observation.\nNEVER conclude that there are no applicable hazards.\n\nOnly return the names of the hazards that are applicable to the observation, \nnever return the descriptions of the hazards.\nThe response should be in JSON format with \'hazard\' as the key and the names \nof all the applicable hazards as the values. \nUse the following format for your response: \n{\'hazard\': [\'Chemical\',\'Safety Systems\']}.\nIf no hazard is applicable return {\'hazard\':[]}\n\nThe names of the hazard categories are as follows.\n\n* "Biological"\n* "Chemical"\n* "Electrical"\n* "Environment"\n* "Gravity"\n* "Human Factors"\n* "Major Accident Hazards"\n* "Mechanical"\n* "Motion"\n* "Pressure"\n* "Radiation"\n* "Safety Systems"\n* "Sound"\n* "Temperature"\n\nRefer to the descriptions of the hazard categories below.\n"NAME OF HAZARD": "Biological"\n"DESCRIPTION OF HAZARD": "Aggressive Animals - Allergies, Bites, Scratches, Kicks, Disease. Bacterial & Viral - Food (Gastro-Intestinal Infections), Air (Cold/Flu) & Water Borne (Legionella), Human/Animal Waste - Gastro-Intestinal Infections (Guano). Decomposing Wildlife - Disease spread (fleas & ticks). Fungal Spores - Mouldy Building Materials, Decaying Vegetable Matter, Rotting Waste Material. "\n"NAME OF HAZARD": "Chemical"\n"DESCRIPTION OF HAZARD": "Airborne Particles - Dust, Fumes, Vapour, Inhalation of fibres (asbestos), Hydrogen Sulphide. Asphyxiation - Nitrogen, Benzene, Halon, Argon, Oxygen Deficiency (Confined Space Work). Carcinogen - Asbestos, Oil, Condensate Vapour, Benzene. Discharge from Vessels - Land, Sea, Air. Flammable - Flashpoint. Hazardous Substance - Benzene, Nitrogen. Irritant - Mineral Fibre (Rockwool), Acid, Caustic. Toxic - Mercury, Chlorine, Hydrogen Sulphide (H2S). "\n"NAME OF HAZARD": "Electrical"\n"DESCRIPTION OF HAZARD": "Electrical Arching - High Current. Electrical Spark - Equipment Condition,\xa0EX Equipment. Equipment - Cables, High/Low Voltage, Non-IS Equipment, Non-standard Modifications. Portable Tools - Serviceability / Maintenance Level. Static - Equipment Condition. Stored/Induced Electrical Charge - Uninterrupted Power Supply. "\n"NAME OF HAZARD": "Environment"\n"DESCRIPTION OF HAZARD": "Emissions - Unauthorised Emissions. Light & Noise Pollution - Local Community Impact, Light Positioning. Spills - Oil and/or Chemical to Deck, Sea or Land, Slips. Waste - NORM, General Waste, Contaminated Material, Segregation, Authorisation. "\n"NAME OF HAZARD": "Gravity"\n"DESCRIPTION OF HAZARD": "Dropped Object/Potential Dropped Object - Deflection, Sliding, Bouncing, Lift Equipment Failure,\xa0Collapse, Stored Energy,\xa0Structural Failure. Falling Person - Suspension Trauma, Safety Equipment Failure, Collapse. Overloading - Shearing, Load Bearing Equipment Failure. Snagging Equipment - Sharp Edges, Protection Failure. "\n"NAME OF HAZARD": "Human Factors"\n"DESCRIPTION OF HAZARD": "Competence Levels - Knowledge, Aptitude, Training, Experience, New/ Unfamiliar Equipment. Design - Legibility, Confusing Signs/ Indications, Complexity. Ergonomics - Body Positioning (Muscular Fatigue), Task Duration, Rests, Equipment, Manual Handling. Human Error Potential - Time Pressure, Milestone Achievement, Job Security, Tiredness, Ill-Health. Individual Capability - Young Persons, Pregnant or Nursing Women, Physical and Mental Health. Lone Working - Communication, Equipment Use, Manual Handling, Emergency Notification. Roles & Responsibilities - Unclear Procedures, Lack of Information, Conflicting Roles. Task - Repetitive, Duration, Attention Required,\xa0Safety Critical. Working Environment - Temperature, Humidity, Ventilation, Light, Posture, Space, Weather. Working Hours - Long Hours, Routine/ Repetitive Work, Fatigue, Prolonged Concentration. "\n"NAME OF HAZARD": "Major Accident Hazards"\n"DESCRIPTION OF HAZARD": "Chemical or Dangerous Substance Release - Uncontrolled release hazardous substances, e.g. H2S or chlorine. Collision with Installation - Ship or aircraft. Dropped Object - Large dropped or swinging object. Explosion - Uncontrolled hydrocarbon release, Uncontrolled release of pressure, unnecessary scaffolding in hydrocarbon-containing areas. Failure or Breach of Safety & Environmental Critical Element (SECE) - E.G. TR integrity compromised (open or damaged door on TR), EER (damaged lifeboat and escape equipment), dive support, shutdown system failure, fire and gas system failure, scaffold blocking F&G heads, loss of dynamic positioning. Fire - Uncontrolled hydrocarbon release with source of ignition, Hot surfaces, Hot work, Non-Ex or faulty Ex equipment, Galley or laundry fire. Hydrocarbon Release - Release from cargo tanks or vessels, leaking flange or pipework or damaged small-bore tubing. Loss of Buoyancy / Stability - Ballasting system failures, Loading and stability errors, Loss of water tight integrity (tank, hull piping failure), Compartment flooding. Loss of Mechanical Integrity - Overloading of lifting equipment or load bearing members, Excessive vibration, Excessive corrosion, Anything else with the potential to cause collapse or weakened integrity. Loss of Well Control - Uncontrolled flow from well bore or annulus, Leak from flexible hoses or small-bore tubing, Failure of any well barrier (mud over-balance, BP, Xmas Tree). Major Environmental Spill to Sea - Leakage of oil or chemicals from pipework, vessels or machinery. Structural Impairment - Strucutural integrity of system or asset compromised through mechanical damage (large dropped or swinging objects, vibration) or corrosion (walkways, handrails, gratings). "\n"NAME OF HAZARD": "Mechanical"\n"DESCRIPTION OF HAZARD": "Equipment Failure - Jacking Equipment, Hand Tools, Doors, Hatches, Gratings, Brackets. Mechanical Handling Equipment - Trolleys, Stands, Surfaces, Weight of Load, Load Route. Rotating Equipment - Serviceability / Maintenance Level. Stored Mechanical Energy - Compressed Springs. "\n"NAME OF HAZARD": "Motion"\n"DESCRIPTION OF HAZARD": "Access & Egress - Congestion, Confined Space, Restricted Movement, Entrapment. Collision Impact - Sharp or Abrasive Object, Trapping, Stored Energy. Discharges - Exhausts, Pipes. Friction - Lubricants. Rotating Equipment - Projectiles, Snagging. Slips, Trips and Falls - Hoses, Pipes, Valves, Stairs, Steps, Excavations. Transport - Aircraft,\xa0Vessels, Vehicles (Forklifts), ROVs. Vibration - HAVS,\xa0Erosion. "\n"NAME OF HAZARD": "Pressure"\n"DESCRIPTION OF HAZARD": "Equipment Failure - Hoses, Small Bore Tubing, Pipework, Air Tools, Valves, Wireline. HP/LP Interfaces - Pressure Regulation. Pressurised Systems - Gas, Compressed Air, Fluid, Trapped Pressure, Bottled Gases, Projectiles, Hoses. Release of Pressure - Over Pressurisation, Pressure Line Release (Whip Checks). Trapped Pressure - Hydrates, Deflections. Vacuum - Pressure Difference. "\n"NAME OF HAZARD": "Radiation"\n"DESCRIPTION OF HAZARD": "Electronic Magnetic Radiation - HV Cables, Switch Rooms. Ionising Radioactive Sources - Nucleonics. Lasers - Positioning (Eye Damage). Microwave & Radiowave - Line of Sight Dishes. NORM - Pipes, Vessels, Valves - Inhalation, Ingestion. UV & IR Radiation - Arc welding, Sunlight/Welding (Skin Burning), Heat Sources (Eye Damage). X-Ray - Positioning (Radiation Exposure). "\n"NAME OF HAZARD": "Safety Systems"\n"DESCRIPTION OF HAZARD": "Area Zone Classification Breach - Explosive Atmospheres\xa0(ATEX) Zone (1, 2, 3 or 20, 21, 22). Safety & Environmentally Critical Element (SECE) Impairment - Communication Equipment, Escape Arrangements, Active Fire Protection, Hydrocarbon Containment, Blowdown or Relief Systems, Drains & Vents, Emergency Power & Lighting, Emergency Shutdown Systems, HVAC, Navigational Aids, PFP, Well Control Equipment, TR, Dynamic Positioning, F&G Detection Systems, Asset Integrity, Diving Systems, Critical Instrumentation. "\n"NAME OF HAZARD": "Sound"\n"DESCRIPTION OF HAZARD": "Excessive Noise - Loss of/ Limited Communication, Long Term Health Effects, Noise Pollution, High or Low Frequency. Vibration - Hand Arm Vibration (HAVS), Duration of Hand Tool Use. "\n"NAME OF HAZARD": "Temperature"\n"DESCRIPTION OF HAZARD": "Cold Surface - Nitrogen Operations, Cold Liquid, Ice. Explosion - Pyrotechnics, Non IS Equipment, Explosives, Pyrophoric Scale, Ignition Source. Fire - Naked Flame, Heat Transfer. Hot Liquid - Oil, Chemical, Flash Point, Flammable Atmosphere. Hot Surface - Heat Transfer. Spontaneous Combustion - Pyrophoric Scale. "Remember,\n(1) The response should be in JSON format with \'hazard\' as the key and a\nlist of all the names of the applicable hazard as the value. \nReturn ONLY the names of the hazard categories as values in a \njson format with \'hazard\' as the key.\nNever return any part of the descriptions of the hazard categories.\n(2) Ensure that the full names of the applicable hazard categories are returned\nin exactly the same way as defined above. Never alter the names of the hazard in any way.\n(3) There must always be at least 1 hazard category associated with each observation.'

In [77]:
lsr_prompt = '''## Task - Life Saving Rules (lsr) Classification

### Your Role:
- You are a process safety expert specializing in high hazard industries.

### Task:
- Classify the provided safety observation into the most appropriate IoGP standard Life Saving Rules (LSRs).
### Instructions:
1. **Categorize the observation** into one or more of the following IoGP standard LSRs based on the descriptions provided.
2. **Output Format:** Generate a JSON object with the following structure:
   ```json
   {
       "lsr": ["category1", "category2"],
       "reasoning": "Short explanation for the classification."
   }
3. **If you are unsure of the classification, return
{
    "lsr": [],
    "reasoning": "Unclear classification."
}

### Categories:

### Instructions:
1. **Categorize the observation** into one or more of the following IoGP standard LSRs based on the descriptions provided.
2. **Output Format:** Generate a JSON object with the following structure:
   ```json
   {
       "lsr": ["category1", "category2"],
       "reasoning": "Short explanation for the classification."
   }
3. **If you are unsure of the classification, return
{
    "lsr": [],
    "reasoning": "Unclear classification."
}

### Categories:


    "Bypassing Safety Control": "Obtain authorization before overriding or disabling safety controls. Safety-critical equipment and procedures must be understood and utilized appropriately. Authorization must be obtained before disabling or overriding safety equipment, deviating from procedures, or crossing barriers. Safety-critical controls include equipment such as fire and explosion protection and mitigation systems, guards, interlocks, alarms, and safety-critical monitoring equipment designed to prevent fatalities or major accidents, limit their consequences, or prevent their substantial contribution to such incidents. Procedures that, if not performed correctly or at the right time, could result in fatalities or major accidents are also included."
    "Confined Space": "Obtain authorization before entering a confined space. Confirm that energy sources are isolated and the atmosphere has been tested and is monitored. Check and use breathing apparatus when required. Ensure an attendant is present, a rescue plan is in place, and authorization to enter is obtained. A confined space, such as a vessel, tank, pipe, cellar, or excavation, can contain explosive gas, toxic or asphyxiating atmosphere, or other dangers such as energy releases, lack of oxygen, exposure to hazardous chemicals, or risks of falling objects or falls. Authorized access ensures safety. Organizations should assess if any work involving excavations or trenches creates confined space conditions and ensure necessary work authorization and controls."
    "Driving Safety": "Follow safe driving rules, which include always wearing a seatbelt, not exceeding the speed limit, reducing speed appropriately for road conditions, not using phones or operating devices while driving, and being fit, rested, and fully alert. Adhere to journey management requirements. The driver and passengers should ensure each other's safety, including seatbelt use. Fitness for duty means ensuring the capability to complete a task safely without unacceptable risk, including not being under the influence of drugs and alcohol."
    "Energy Isolation": "Verify isolation and zero energy before work begins. Identify all energy sources, confirm hazardous energy sources are isolated, locked, and tagged, and check for zero energy and residual or stored energy. Energy isolation separates people from hazards such as electricity, pressure, and energized equipment, and provides protection from potential energy sources like gravity-fed tanks. Any stored energy (e.g., hydraulic or pneumatic power) should be dissipated before starting work."
    "Hot Work": "Control flammables and ignition sources. I identify and control ignition sources. Before starting any hot work I confirm flammable material has been removed or isolated, I obtain authorisation. Before starting hot work in a hazardous area I confirm that a gas test has been completed, and gas will be monitored continually. Hot work includes any work that creates an ignition source performed in an area which potentially contains hydrocarbons or flammable materials. Ignition sources are open flames or sources of heat that could ignite materials in the work area such as welding, grinding, smoking, torching, (un)loading of hazardous materials, internal combustion engines, chemical reactions, batteries, etc. Hazardous areas are defined in the UK Dangerous Substances and Explosive Atmospheres Regulations 2002 (DSEAR) as any place in which an explosive atmosphere may occur in quantities such as to require special precautions to protect the safety of workers."

   "Line of Fire": "Keep yourself and others out of the line of fire. I position myself to avoid moving objects, vehicles, pressure releases, dropped objects. I establish and obey barriers and exclusion zones. I take action to secure loose objects and report potential dropped objects. Other rules focus on specific activities, whereas this Rule is intended to raise personal awareness of struck-by and caught in-between hazards. Line of fire hazards are not always obvious or constant and can be introduced as the task progresses (e.g. underground and overhead powerlines, pipelines, objects under pressure, stored energy, lines under tension, poorly supported excavations, shifting cargo, moving equipment). At all times individuals need to continually monitor their surroundings and position themselves to avoid being in the line of fire. This includes ensuring they are visible to vehicle drivers and equipment operators. Individuals recognise when they create a line of fire hazard and put others or themselves in the line of fire"
    "Safe Mechanical Lifting": "Plan lifting operations and control the area. I confirm that the equipment and load have been inspected and are fit for purpose. I only operate equipment that I am qualified to use. I establish and obey barriers and exclusion zones. I never walk under a suspended load. Lifting operations need to be planned and performed by competent personnel using certified equipment. To protect people around suspended loads and any mechanical lifting operation, access should be controlled through physical barriers and exclusion zones. When mechanically lifting people (e.g. manriding, man baskets, personnel transfer, mobile elevated work platform), organisations should provide equipment which is designed and certified specifically for lifting people."
    "Work Authorisation": "Work with a valid permit when required. I have confirmed if a permit is required. I am authorised to perform the work. I understand the permit. I have confirmed that hazards are controlled and it is safe to start. I stop and reassess if conditions change. Work authorisation is more than just a person in charge signing a Permit to Work form: it is seeking and having authorisation to start, resume, or hand-over a task. The person in charge of the work confirms that it is safe to start, that controls are in place and effective, and the task can be performed as planned."
    "Working at Height": "Protect yourself against a fall when working at height. I inspect my fall protection equipment before use. I secure tools and work materials to prevent dropped objects. I tie off 100% to approved anchor points while outside a protected area. Working at height outside a protected area (such as an elevated work area not enclosed by hand rails) requires the use of approved fall protection equipment secured to an approved anchor point. Other considerations for working at height include ladders, work over water, rope access, floor openings, access hatches, and inspection pits. Floor openings should be protected with physical barriers to prevent falls. Preventing objects from falling from height and using physical barriers below working area keeps you and people working below you safe. IOGP recommends that companies define working at height as work at or above 1.8m/6ft, unless local legislation requires a lower height. Scaffolds should be properly constructed, inspected and certified."


### Additional Context:
"No Smoking" is covered under "Hot Work."
"Safe Mechanical Lifting" does not include helicopter lifts.
- **Note:** "Bypassing Safety Control" and other LSRs apply to intentional human actions and behaviors. Hardware malfunctions or failures should not be classified under these categories.

## Examples ##

observation: When reviewing worksites, OP was not wearing hearing protection whilst working
{'lsr': ['Bypassing Safety Control'], 'reasoning': 'OP has not obtained authorisation to not wear hearing protection as it is a requirement. Therefore this LSR is Bypassing Safety Control as the OP is deviating from procedures.'}

observation: SUB STANDARD - BLP Crane Anemometer inoperable and has been for past year, this is a critical piece of equipment during crane operations if weather borderline or on the rise
{'lsr': ['Safe Mechanical Lifting'], 'reasoning': 'A safety critical piece of equipment for safe mechanical lifting using a crane is not working. The lifting equipment is therefore unsafe to use and the LSR is Safe Mechanical Lifting.'}

observation: Whilst reviewing a previously amended ICC it was noticed that a 2nd IA review hadn't taken place before its Isolation in progress phase
{'lsr': ['Work Authorisation'], 'reasoning': 'Work authorisation definition states that all permits are valid and complete before work starts and that it is confirmed that hazards are controlled and it is safe to start.'}

observation: Rope access team were carrying out a flood light replacement on the Turret level 2. When making way up to the worksite one of the rope access techs made note of the poor state of the access ladder back scratcher. The back scratcher was severely corroded to the point it could result in a PDO.
{'lsr': ['Working at Height'], 'reasoning': 'Working from Height definition states to secure yourself against a fall when working at height by ensuring equipment is safe. It has been observed that the equipment for safe work at height has degraded. This would increase the chances of a fall from height.'}

observation: 14'x 20' steel packers were in an untidy mess next to a structural beam
{'lsr': [], 'reasoning': 'No LSR is applicable in this case'}

observation: Observed team working on the BLP. Part of the task involved working at height using either fall restraint or fall arrest depending on the location/task. Reviewing the worksite a number of vertical scaffold tubes were observed protruding well above the ledger of a scaffold located below the W@H location introducing an impalement hazard.
{'lsr': ['Line of Fire', 'Working at Height'], 'reasoning': "The task involved working at height but also there was potential for the vertical scaffold tubes to fall. Part of Line of Fire's definition is to avoid moving objects and also dropped objects."}

observation: On discovery of a leak on FWP 'A', I brought forward an issue identified yesterday on FWP 'C'. One of FWP 'C' suction valves is showing in fault and in the closed position within ICSS. The valve is situated within a confined space within the column and so not easily accessible to confirm the position.
{'lsr': ['Bypassing Safety Control'], 'reasoning': 'A suction valve has been purposely closed which is bypassing the safety control that it should be in.'}
 '''

In [83]:
LOGPROB_FLAG = True
TOP_LOGPROB_FLAG = True
N_TOP_LOGPROB = 3

message_text = [
    {"role": "system", "content": lsr_prompt},
    {"role": "user", "content": obs2}
]

if TOP_LOGPROB_FLAG and LOGPROB_FLAG:
    completion = client.chat.completions.create(
        model=os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT'),
        messages=message_text,
        temperature=0.2,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=None,
        logprobs=LOGPROB_FLAG,
        top_logprobs=N_TOP_LOGPROB,
        response_format={"type": "json_object"}
    )
else:
    completion = client.chat.completions.create(
        model=os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT'),
        messages=message_text,
        temperature=0.2,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=None,
        logprobs=LOGPROB_FLAG,
        response_format={"type": "json_object"}
    )

In [84]:
print("Message: \n")
print_message(completion)
print("---------------\n")
if LOGPROB_FLAG:
    print_lp_info(completion)

Message: 

{
    "lsr": [],
    "reasoning": "No LSR is applicable in this case"
}
---------------

'Token: `{\n`, Logprob: -0.0030677915 (99.69%)'
'Token: `   `, Logprob: -0.03363509 (96.69%)'
'Token: ` "`, Logprob: -3.1281633e-07 (100.0%)'
'Token: `ls`, Logprob: -2.6968896e-06 (100.0%)'
'Token: `r`, Logprob: -3.1281633e-07 (100.0%)'
'Token: `":`, Logprob: -8.852353e-05 (99.99%)'
'Token: ` [],\n`, Logprob: -0.27410632 (76.03%)'
'Token: `   `, Logprob: -0.00016361084 (99.98%)'
'Token: ` "`, Logprob: 0.0 (100.0%)'
'Token: `reason`, Logprob: -8.776276e-06 (100.0%)'
'Token: `ing`, Logprob: -1.0280384e-06 (100.0%)'
'Token: `":`, Logprob: -8.804671e-05 (99.99%)'
'Token: ` "`, Logprob: -7.3458323e-06 (100.0%)'
'Token: `No`, Logprob: -0.64011824 (52.72%)'
'Token: ` L`, Logprob: -0.25587404 (77.42%)'
'Token: `SR`, Logprob: -2.4584822e-06 (100.0%)'
'Token: ` is`, Logprob: -0.0039620437 (99.6%)'
'Token: ` applicable`, Logprob: -0.024249898 (97.6%)'
'Token: ` in`, Logprob: -0.008314914 (99.17%)'


In [85]:
print_top_lp(completion)

'Token: `{\n`, Logprob: -0.0030677915 (99.69%)'
Top Logprobs:
'    `{\n`: -0.0030677915 (99.69%)'
'    `\n`: -5.982913 (0.25%)'
'    `{"`: -7.6985254 (0.05%)'
'Token: `   `, Logprob: -0.03363509 (96.69%)'
Top Logprobs:
'    `   `: -0.03363509 (96.69%)'
'    ` `: -3.4284754 (3.24%)'
'    `  `: -7.5695543 (0.05%)'
'Token: ` "`, Logprob: -3.1281633e-07 (100.0%)'
Top Logprobs:
'    ` "`: -3.1281633e-07 (100.0%)'
'    `\t`: -15.716017 (0.0%)'
'    ` "_`: -17.358448 (0.0%)'
'Token: `ls`, Logprob: -2.6968896e-06 (100.0%)'
Top Logprobs:
'    `ls`: -2.6968896e-06 (100.0%)'
'    `LS`: -13.499976 (0.0%)'
'    `reason`: -14.159507 (0.0%)'
'Token: `r`, Logprob: -3.1281633e-07 (100.0%)'
Top Logprobs:
'    `r`: -3.1281633e-07 (100.0%)'
'    `rs`: -16.097769 (0.0%)'
'    `rr`: -17.523573 (0.0%)'
'Token: `":`, Logprob: -8.852353e-05 (99.99%)'
Top Logprobs:
'    `":`: -8.852353e-05 (99.99%)'
'    `":["`: -9.645255 (0.01%)'
'    `"`: -10.781222 (0.0%)'
'Token: ` [],\n`, Logprob: -0.27410632 (76.03%)'
Top

In [86]:
lps = extract_lp(completion)
av_lp = np.mean(lps)

print(lp2p(av_lp))

92.38


In [112]:
from pprint import pp

pp(completion.choices[0].logprobs.content[0].__dict__)

{'token': '{\n',
 'bytes': [123, 10],
 'logprob': -0.0030677915,
 'top_logprobs': [TopLogprob(token='{\n', bytes=[123, 10], logprob=-0.0030677915),
                  TopLogprob(token='\n', bytes=[10], logprob=-5.982913),
                  TopLogprob(token='{"', bytes=[123, 34], logprob=-7.6985254)]}


In [130]:
output = {}

for key, value in lp[0].__dict__.items():
    if key == 'token':
        output[value] = {}
        temp = value
    elif key == 'logprob':
        output[temp][key] = value
    elif key == 'top_logprobs':
        output[temp][key] = [(x.token, x.logprob) for x in value]

In [131]:
print(output)

{'{\n': {'logprob': -0.0030677915, 'top_logprobs': [('{\n', -0.0030677915), ('\n', -5.982913), ('{"', -7.6985254)]}}


In [129]:
lp[0].__dict__['top_logprobs']

t = [(x.token, x.logprob) for x in lp[0].__dict__['top_logprobs']]

print(t)

[('{\n', -0.0030677915), ('\n', -5.982913), ('{"', -7.6985254)]


In [133]:
lp = completion.choices[0].logprobs.content

output = {}
temp = None

for i in lp:
    for key, value in i.__dict__.items():
        if key == 'token':
            output[value] = {}
            temp = value
        elif key == 'logprob':
            output[temp][key] = value
        elif key == 'top_logprobs':
            output[temp][key] = [(x.token, x.logprob) for x in value]

pp(output)

{'{\n': {'logprob': -0.0030677915,
         'top_logprobs': [('{\n', -0.0030677915),
                          ('\n', -5.982913),
                          ('{"', -7.6985254)]},
 '   ': {'logprob': -0.00016361084,
         'top_logprobs': [('   ', -0.00016361084),
                          ('  ', -9.826618),
                          ('\t', -9.950138)]},
 ' "': {'logprob': -7.3458323e-06,
        'top_logprobs': [(' "', -7.3458323e-06),
                         (' "\'', -12.124529),
                         (' ', -13.290581)]},
 'ls': {'logprob': -2.6968896e-06,
        'top_logprobs': [('ls', -2.6968896e-06),
                         ('LS', -13.499976),
                         ('reason', -14.159507)]},
 'r': {'logprob': -3.1281633e-07,
       'top_logprobs': [('r', -3.1281633e-07),
                        ('rs', -16.097769),
                        ('rr', -17.523573)]},
 '":': {'logprob': -8.804671e-05,
        'top_logprobs': [('":', -8.804671e-05),
                         ('":"', 